In [2]:
# -*- coding:utf-8 -*-
# @FileName : gmm_hmm.py
# @Time : 2024/4/9 19:17
# @Author : fiv
# Isolated word recognition

import librosa
import numpy as np
from sklearn.cluster import KMeans
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

def extract_MFCC(wav_file):
    # 读取音频文件
    y, sr = librosa.load(wav_file, sr=8000)
    # 提取特征
    fea = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12, n_mels=24, n_fft=256, win_length=256, hop_length=80, lifter=12)
    # 进行正则化
    mean = np.mean(fea, axis=1, keepdims=True)
    std = np.std(fea, axis=1, keepdims=True)
    fea = (fea - mean) / std
    # 添加一阶差分
    fea_d = librosa.feature.delta(fea)
    fea = np.concatenate([fea.T, fea_d.T], axis=1)
    return fea


def init_para_hmm(collect_fea, N_state, N_mix):
    # 初始一定从state=0开始
    pi = np.zeros(N_state)
    pi[0] = 1
    # 当前状态转移概率0.5，下一状态转移概率0.5
    # 进入最后一格状态后不再跳出
    A = np.zeros([N_state, N_state])
    for i in range(N_state - 1):
        A[i, i] = 0.5
        A[i, i + 1] = 0.5
    A[-1, -1] = 1
    feas = collect_fea
    len_feas = []
    for fea in feas:
        len_feas.append(np.shape(fea)[0])
    _, D = np.shape(feas[0])
    hmm_means = np.zeros([N_state, N_mix, D])
    hmm_sigmas = np.zeros([N_state, N_mix, D])
    hmm_ws = np.zeros([N_state, N_mix])
    for s in range(N_state):
        sub_fea_collect = []
        for fea, T in zip(feas, len_feas):
            T_s = int(T / N_state) * s
            T_e = (int(T / N_state)) * (s + 1)
            sub_fea_collect.append(fea[T_s:T_e])
        ws, mus, sigmas = gen_para_GMM(sub_fea_collect, N_mix)
        hmm_means[s] = mus
        hmm_sigmas[s] = sigmas
        hmm_ws[s] = ws
    return pi, A, hmm_means, hmm_sigmas, hmm_ws


def run_kmeans(dataset, K, m=20):
    labs = KMeans(n_clusters=K, random_state=9).fit_predict(dataset)  # 聚成K类
    return labs


def gen_para_GMM(fea_collect, N_mix):
    # 首先对特征进行kmeans聚类
    feas = np.concatenate(fea_collect, axis=0)
    N, D = np.shape(feas)
    # 初始化聚类中心
    labs = run_kmeans(feas, N_mix, m=20)
    mus = np.zeros([N_mix, D])
    sigmas = np.zeros([N_mix, D])
    ws = np.zeros(N_mix)
    for m in range(N_mix):
        index = np.where(labs == m)[0]
        sub_feas = feas[index]
        mu = np.mean(sub_feas, axis=0)
        sigma = np.var(sub_feas, axis=0)
        sigma = sigma + 0.0001
        mus[m] = mu
        sigmas[m] = sigma
        ws[m] = np.shape(index)[0] / N
    ws = (ws + 0.01) / np.sum(ws + 0.01)
    return ws, mus, sigmas


import os
from hmmlearn.hmm import GMMHMM
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

if __name__ == "__main__":
    models = []
    train_path = "../../data/isolated_word/train"
    for i in range(1, 15):
        # 进入孤立词所在的文件夹
        wav_path = os.path.join(train_path, str(i))
        collect_fea = []
        len_feas = []
        dirs = os.listdir(wav_path)
        for file in dirs:
            # 找到 .wav文件并提取特征
            if file.split(".")[-1] == "wav":
                wav_file = os.path.join(wav_path, file)
                fea = extract_MFCC(wav_file)
                collect_fea.append(fea)
                len_feas.append(np.shape(fea)[0])
        # 获取模型参数初始化
        N_state = 4
        N_mix = 3
        pi, A, hmm_means, hmm_sigmas, hmm_ws = init_para_hmm(collect_fea, N_state, N_mix)
        train_GMMHMM = GMMHMM(n_components=N_state,
                              n_mix=N_mix,
                              covariance_type='diag',
                              n_iter=90,
                              tol=1e-5,
                              verbose=False,
                              init_params="",
                              params="tmcw",
                              min_covar=0.0001
                              )
        train_GMMHMM.startprob_ = pi
        train_GMMHMM.transmat_ = A
        train_GMMHMM.weights_ = hmm_ws
        train_GMMHMM.means_ = hmm_means
        train_GMMHMM.covars_ = hmm_sigmas
        print("train GMM-GMM", i)
        train_GMMHMM.fit(np.concatenate(collect_fea, axis=0), np.array(len_feas))
        models.append(train_GMMHMM)
    np.save("models_hmmlearn.npy", models)

# 测试部分
test_dir = "../../data/isolated_word/test"
models = np.load("models_hmmlearn.npy", allow_pickle=True)
count = 0
count2 = 0
for i in range(98):
    # 读取wav文件
    wav_file = os.path.join(test_dir, str(i + 1) + ".wav")
    fea = extract_MFCC(wav_file)
    lab_true = int(i // 7) + 1
    scores = []
    scores2 = []
    for m in range(1, 15):
        model = models[m - 1]
        score, _ = model.decode(fea)
        scores.append(score)
        score2 = model.score(fea)
        scores2.append(score2)

    det_lab = np.argmax(scores) + 1
    det_lab2 = np.argmax(scores2) + 1
    if det_lab == lab_true:
        count = count + 1
    if det_lab2 == lab_true:
        count2 = count2 + 1
    print("true lab %d det lab1 %d det lab2 %d" % (lab_true, det_lab, det_lab2))
print("decode %.2f     " % (count * 100 / 98))

V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

train GMM-GMM 1


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

train GMM-GMM 2


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

...

train GMM-GMM 13


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

train GMM-GMM 14


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 2 det lab2 2
true lab 1 det lab1 1 det lab2 1
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 2 det lab2 2
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 5 det lab1 5 det lab2 5
true lab 5 det lab1 5 det lab2 5
true lab 5

In [3]:
# -*- coding:utf-8 -*-
# @FileName : gmm_hmm.py
# @Time : 2024/4/9 19:17
# @Author : fiv
# Isolated word recognition

import librosa
import numpy as np
from sklearn.cluster import KMeans
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


def extract_MFCC(wav_file):
    # 读取音频文件
    y, sr = librosa.load(wav_file, sr=8000)
    # 提取特征
    fea = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12, n_mels=24, n_fft=256, win_length=256, hop_length=80, lifter=12)
    # 进行正则化
    mean = np.mean(fea, axis=1, keepdims=True)
    std = np.std(fea, axis=1, keepdims=True)
    fea = (fea - mean) / std
    # 添加一阶差分
    fea_d = librosa.feature.delta(fea)
    fea = np.concatenate([fea.T, fea_d.T], axis=1)
    return fea


def init_para_hmm(collect_fea, N_state, N_mix):
    # 初始一定从state=0开始
    pi = np.zeros(N_state)
    pi[0] = 1
    # 当前状态转移概率0.5，下一状态转移概率0.5
    # 进入最后一格状态后不再跳出
    A = np.zeros([N_state, N_state])
    for i in range(N_state - 1):
        A[i, i] = 0.5
        A[i, i + 1] = 0.5
    A[-1, -1] = 1
    feas = collect_fea
    len_feas = []
    for fea in feas:
        len_feas.append(np.shape(fea)[0])
    _, D = np.shape(feas[0])
    hmm_means = np.zeros([N_state, N_mix, D])
    hmm_sigmas = np.zeros([N_state, N_mix, D])
    hmm_ws = np.zeros([N_state, N_mix])
    for s in range(N_state):
        sub_fea_collect = []
        for fea, T in zip(feas, len_feas):
            T_s = int(T / N_state) * s
            T_e = (int(T / N_state)) * (s + 1)
            sub_fea_collect.append(fea[T_s:T_e])
        ws, mus, sigmas = gen_para_GMM(sub_fea_collect, N_mix)
        hmm_means[s] = mus
        hmm_sigmas[s] = sigmas
        hmm_ws[s] = ws
    return pi, A, hmm_means, hmm_sigmas, hmm_ws


def run_kmeans(dataset, K, m=20):
    labs = KMeans(n_clusters=K, random_state=9).fit_predict(dataset)  # 聚成K类
    return labs


def gen_para_GMM(fea_collect, N_mix):
    # 首先对特征进行kmeans聚类
    feas = np.concatenate(fea_collect, axis=0)
    N, D = np.shape(feas)
    # 初始化聚类中心
    labs = run_kmeans(feas, N_mix, m=20)
    mus = np.zeros([N_mix, D])
    sigmas = np.zeros([N_mix, D])
    ws = np.zeros(N_mix)
    for m in range(N_mix):
        index = np.where(labs == m)[0]
        sub_feas = feas[index]
        mu = np.mean(sub_feas, axis=0)
        sigma = np.var(sub_feas, axis=0)
        sigma = sigma + 0.0001
        mus[m] = mu
        sigmas[m] = sigma
        ws[m] = np.shape(index)[0] / N
    ws = (ws + 0.01) / np.sum(ws + 0.01)
    return ws, mus, sigmas


import os
from hmmlearn.hmm import GMMHMM
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

if __name__ == "__main__":
    models = []
    train_path = "../../data/isolated_word/train"
    for i in range(1, 15):
        # 进入孤立词所在的文件夹
        wav_path = os.path.join(train_path, str(i))
        collect_fea = []
        len_feas = []
        dirs = os.listdir(wav_path)
        for file in dirs:
            # 找到 .wav文件并提取特征
            if file.split(".")[-1] == "wav":
                wav_file = os.path.join(wav_path, file)
                fea = extract_MFCC(wav_file)
                collect_fea.append(fea)
                len_feas.append(np.shape(fea)[0])
        # 获取模型参数初始化
        N_state = 4
        N_mix = 3
        pi, A, hmm_means, hmm_sigmas, hmm_ws = init_para_hmm(collect_fea, N_state, N_mix)
        train_GMMHMM = GMMHMM(n_components=N_state,
                              n_mix=N_mix,
                              covariance_type='diag',
                              n_iter=90,
                              tol=1e-5,
                              verbose=False,
                              init_params="",
                              params="tmcw",
                              min_covar=0.0001
                              )
        train_GMMHMM.startprob_ = pi
        train_GMMHMM.transmat_ = A
        train_GMMHMM.weights_ = hmm_ws
        train_GMMHMM.means_ = hmm_means
        train_GMMHMM.covars_ = hmm_sigmas
        print("train GMM-GMM", i)
        train_GMMHMM.fit(np.concatenate(collect_fea, axis=0), np.array(len_feas))
        models.append(train_GMMHMM)
    np.save("models_hmmlearn.npy", models)

# 测试部分
test_dir = "../../data/isolated_word/test"
models = np.load("models_hmmlearn.npy", allow_pickle=True)
count = 0
count2 = 0
for i in range(98):
    # 读取wav文件
    wav_file = os.path.join(test_dir, str(i + 1) + ".wav")
    fea = extract_MFCC(wav_file)
    lab_true = int(i // 7) + 1
    scores = []
    scores2 = []
    for m in range(1, 15):
        model = models[m - 1]
        score, _ = model.decode(fea)
        scores.append(score)
        score2 = model.score(fea)
        scores2.append(score2)

    det_lab = np.argmax(scores) + 1
    det_lab2 = np.argmax(scores2) + 1
    if det_lab == lab_true:
        count = count + 1
    if det_lab2 == lab_true:
        count2 = count2 + 1
    print("true lab %d det lab1 %d det lab2 %d" % (lab_true, det_lab, det_lab2))
print("decode %.2f     " % (count * 100 / 98))

V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

train GMM-GMM 1


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

train GMM-GMM 2


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

...

train GMM-GMM 13


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Wi

train GMM-GMM 14


V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
V:\ENV\mambaforge\envs\vp\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 1 det lab2 1
true lab 1 det lab1 2 det lab2 2
true lab 1 det lab1 1 det lab2 1
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 2 det lab1 2 det lab2 2
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 3 det lab1 3 det lab2 3
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 2 det lab2 2
true lab 4 det lab1 4 det lab2 4
true lab 4 det lab1 4 det lab2 4
true lab 5 det lab1 5 det lab2 5
true lab 5 det lab1 5 det lab2 5
true lab 5